In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks/Hackerearth Challenges/Deeplearning 1')

In [ ]:
pwd

'/content/drive/My Drive/Colab Notebooks/Hackerearth Challenges/Deeplearning 1'

In [ ]:
import pandas as pd
from keras.preprocessing import image
import matplotlib.pyplot as plt

In [ ]:
import os
import numpy as np

In [ ]:
# !wget -O data t.ly/Wh06

In [ ]:
# !unzip data

In [ ]:
df = pd.read_csv('./dataset/train.csv')
df.head()

,Image,Class
0,image3476.jpg,Miscellaneous
1,image5198.jpg,Candle
2,image4183.jpg,Snowman
3,image1806.jpg,Miscellaneous
4,image7831.jpg,Miscellaneous


In [ ]:
df['Class'].value_counts()

Miscellaneous     2801
Christmas_Tree    1539
Jacket             640
Candle             593
Airplane           535
Snowman            361
Name: Class, dtype: int64

In [ ]:
mapp = {}
rev_mapp = {}
curr = 0
for label in df['Class'].unique():
    mapp[label] = curr
    rev_mapp[curr] = label
    curr += 1

In [ ]:
temp_data = {}
for index, row in df.iterrows():
    temp_data[row['Image']] = row['Class']

In [ ]:
# temp_data

In [ ]:
os.listdir()

['weights.h5',
 'data',
 'BasicCNN.ipynb',
 'Transfer learning.ipynb',
 'dataset',
 'out.csv']

## Training set

In [ ]:
ih = 150
iw = 130

In [ ]:
path = './dataset/train'
X_train = []
y_train = []

In [ ]:
from tqdm import tqdm
for file in tqdm(os.listdir(path)):
    img = image.load_img(os.path.join(path, file), target_size = (ih,iw))
    x = image.img_to_array(img)
    X_train.append(x)
    y_train.append(mapp[temp_data[file]])

100%|██████████| 6469/6469 [00:07<00:00, 836.44it/s]


In [ ]:
X_train = np.stack(X_train)
y_train = np.stack(y_train)

In [ ]:
print(X_train.shape, y_train.shape)

(6469, 150, 130, 3) (6469,)


In [ ]:
from sklearn.model_selection import train_test_split
XX_train, XX_test, yy_train, yy_test = train_test_split(X_train, y_train, test_size = 0.05)

In [ ]:
print(XX_train.shape, XX_test.shape)

(6145, 150, 130, 3) (324, 150, 130, 3)


In [ ]:
path = './dataset/test'
test_X = []
file_name = []

In [ ]:
for file in tqdm(os.listdir(path)):
    file_name.append(file)
    img = image.load_img(os.path.join(path, file), target_size = (ih, iw))
    x = image.img_to_array(img)
    test_X.append(x)

100%|██████████| 3489/3489 [00:04<00:00, 841.90it/s]


In [ ]:
test_X = np.stack(test_X)

In [ ]:
test_X.shape

(3489, 150, 130, 3)

In [ ]:
import tensorflow as tf

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = {}):
        if(logs.get('acc') >= .98):
            print("Cancelling Training")
            self.model.stop_training = True

In [ ]:
callbacks = myCallback()

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32, (3,3), activation='tanh', input_shape=(ih, iw, 3)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='tanh'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(128, (3,3), activation='tanh'),
  tf.keras.layers.AveragePooling2D(2,2),
  tf.keras.layers.Conv2D(256, (3,3), activation='tanh'),
  tf.keras.layers.AveragePooling2D(2,2),
   tf.keras.layers.Conv2D(512, (3,3), activation='tanh'),
  tf.keras.layers.AveragePooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='tanh'),
  tf.keras.layers.Dense(1024, activation='tanh'),
  tf.keras.layers.Dense(6, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
model.summary()
# model.fit(X_train, y_train, epochs = 50, batch_size = 32, callbacks = [callbacks])
model.fit(XX_train, yy_train, epochs = 50, batch_size = 16, validation_data = (XX_test, yy_test), callbacks = [callbacks])

385/385 [==============================] - 6s 15ms/step - loss: 1.6233 - acc: 0.4138 - val_loss: 1.6382 - val_acc: 0.4290
Epoch 28/50
385/385 [==============================] - 6s 15ms/step - loss: 1.6613 - acc: 0.3866 - val_loss: 1.6339 - val_acc: 0.4290
Epoch 29/50
385/385 [==============================] - 6s 15ms/step - loss: 1.6504 - acc: 0.3971 - val_loss: 1.6985 - val_acc: 0.4290
Epoch 30/50
385/385 [==============================] - 6s 15ms/step - loss: 1.6391 - acc: 0.3918 - val_loss: 1.7723 - val_acc: 0.0864
Epoch 31/50
329/385 [========================>.....] - ETA: 0s - loss: 1.6851 - acc: 0.3699

## Prediction

In [ ]:
y_hat = model.predict(test_X)

In [ ]:
test_y = []

In [ ]:
for lis in y_hat:
    index = np.argmax(lis)
    test_y.append(rev_mapp[index])

In [ ]:
print(test_y[:5])

['Jacket', 'Candle', 'Jacket', 'Christmas_Tree', 'Miscellaneous']


In [ ]:
res = {}
for i in range(test_X.shape[0]):
    res[file_name[i]] = test_y[i]

In [ ]:
result = pd.DataFrame(list(res.items()), columns = ['Image', 'Class'])

In [ ]:
result.head()

,Image,Class
0,image8132.jpg,Jacket
1,image1490.jpg,Candle
2,image3111.jpg,Jacket
3,image664.jpg,Christmas_Tree
4,image5802.jpg,Miscellaneous


In [ ]:
print(result.head())

           Image           Class
0  image8132.jpg          Jacket
1  image1490.jpg          Candle
2  image3111.jpg          Jacket
3   image664.jpg  Christmas_Tree
4  image5802.jpg   Miscellaneous


In [ ]:
result.to_csv('out.csv',index = False)

In [ ]:
dff = pd.read_csv('out.csv')

In [ ]:
dff.head()

,Image,Class
0,image8132.jpg,Miscellaneous
1,image1490.jpg,Candle
2,image3111.jpg,Jacket
3,image664.jpg,Christmas_Tree
4,image5802.jpg,Miscellaneous
